In [2]:
import glob
import pickle
import os
import sys

if not os.path.join(os.path.abspath(''), '../') in sys.path:
    sys.path.append(os.path.join(os.path.abspath(''), '../'))
    
import numpy as np
import pandas as pd
from tqdm import notebook

from utils.feature_utils import filter_pose_noise
from utils.meta import *

### specific to where I stored the csvs. you can change the data_root_dir

In [3]:
data_root_dir = '/Users/alexanderhsu/Documents/ACC_Study3_Habituation'
# sexes = ['Male', 'Female']
# groups = ['A_Baseline_NoSNI', 'B_Baseline_SNI', 'C_3WSNI_DCZ', 'D_4WSNI_DCZ']

# groups = [f'Group{i}' for i in range(1, 8)]

# conditions = [f'Condition{i}' for i in range(1, 4)]

# conditions_extra = ['Condition3A', 'Condition3B']

### store all data in dictionary

In [4]:

filenames = {key: [] for key in conditions_hab}
data = {key: [] for key in conditions_hab}
for condition in notebook.tqdm(conditions_hab):
    
    filenames[condition] = glob.glob(str.join('/', 
                                                          (data_root_dir,
                                                           f'{condition}', 
                                                           '*.csv')))
    data[condition] = {key: [] for key in [f'file{i}' 
                                                       for i in range(len(filenames[condition]))]}
    for i, csv in enumerate(notebook.tqdm(filenames[condition])):
        temp_df = pd.read_csv(csv, header=[0, 1, 2, 3], sep=",", index_col=0)
        selected_pose_idx = np.arange(temp_df.shape[1])
        idx_llh = selected_pose_idx[2::3]
        # the loaded sleap file has them too, so exclude for both
        idx_selected = [i for i in selected_pose_idx if i not in idx_llh]
        currdf_filt, _ = filter_pose_noise(temp_df, idx_selected=idx_selected, idx_llh=idx_llh, 
                                           llh_value=0.1)
        data[condition][f'file{i}'] = currdf_filt

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [5]:
file_chosen = 0
filenames['HAB_D1_Male'][file_chosen]

'/Users/alexanderhsu/Documents/ACC_Study3_Habituation/HAB_D1_Male/D1_2088.3_M_L1DLC_resnet50_LUPE_MALEDec5shuffle1_350000.csv'

In [6]:
data['HAB_D1_Male'][f'file{file_chosen}']

array([[107.01745998, 224.94100642, 113.44081473, ..., 408.48378611,
        253.00772476, 439.37444925],
       [118.38655806, 222.60738635, 123.6037128 , ..., 405.87756538,
        251.37290943, 438.80448771],
       [133.61788559, 221.2691431 , 137.89443928, ..., 410.68648028,
        252.63523054, 443.25323629],
       ...,
       [569.40780368, 241.35123622, 572.95037866, ..., 345.85917008,
        596.60557652, 399.7288177 ],
       [572.77201176, 241.89855313, 578.82263803, ..., 345.1069113 ,
        596.89023113, 400.09208465],
       [576.61872187, 243.81083488, 579.26020575, ..., 343.33890033,
        597.14249849, 394.79957464]])

In [7]:
with open('../processed_dataset/raw_data_hab.pkl', 'wb') as f:
    pickle.dump(data, f)